### Download Python Library

In [58]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense,Activation,Dropout
from keras.optimizers import SGD
import random

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rohit.pratapwar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\rohit.pratapwar\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Initializing Chatbot Training

In [59]:
words = []
classes = []
documents = []
ingore_words = ['?','!','#','$']
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [60]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # adding documents
        documents.append((w, intent['tag']))

        # adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [61]:

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ingore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")

print (len(classes), "classes", classes)

print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

47 documents
9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
88 unique lemmatized words ["'s", ',', 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']


### Building the Deep Learning Model

In [62]:
# initializing training data

training = []

output_empty = [0] * len(classes)

for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
        
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag,output_row])
    
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training Data Successfully created")

Training Data Successfully created


<ipython-input-62-a7e1596fb32a>:26: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  training = np.array(training)


In [63]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax

model = Sequential()
model.add(Dense(128,input_shape=(len(train_x[0]),),activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]),activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model

sgd = SGD(lr=0.01,decay=1e-6,momentum=0.9,nesterov=True)

model.compile(loss='categorical_crossentropy',optimizer = sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x),np.array(train_y),epochs=250,batch_size=5,verbose=1)

model.save('chatbot_model.h5',hist)

print("Model Successfully Created")


Epoch 1/250
10/10 [==============================] - 0s 1ms/step - loss: 2.2614 - accuracy: 0.0426
Epoch 2/250
10/10 [==============================] - 0s 881us/step - loss: 2.1739 - accuracy: 0.1277
Epoch 3/250
10/10 [==============================] - 0s 897us/step - loss: 2.1188 - accuracy: 0.1489
Epoch 4/250
10/10 [==============================] - 0s 767us/step - loss: 2.0039 - accuracy: 0.2979
Epoch 5/250
10/10 [==============================] - 0s 799us/step - loss: 1.9669 - accuracy: 0.4043
Epoch 6/250
10/10 [==============================] - 0s 997us/step - loss: 1.7817 - accuracy: 0.4043
Epoch 7/250
10/10 [==============================] - 0s 897us/step - loss: 1.7047 - accuracy: 0.4894
Epoch 8/250
10/10 [==============================] - 0s 898us/step - loss: 1.6232 - accuracy: 0.4681
Epoch 9/250
10/10 [==============================] - 0s 798us/step - loss: 1.3419 - accuracy: 0.6383
Epoch 10/250
10/10 [==============================] - 0s 904us/step - loss: 1.3716 - accuracy

10/10 [==============================] - 0s 698us/step - loss: 0.0699 - accuracy: 0.9787
Epoch 82/250
10/10 [==============================] - 0s 812us/step - loss: 0.1407 - accuracy: 0.9362
Epoch 83/250
10/10 [==============================] - 0s 799us/step - loss: 0.0642 - accuracy: 1.0000
Epoch 84/250
10/10 [==============================] - 0s 709us/step - loss: 0.0331 - accuracy: 1.0000
Epoch 85/250
10/10 [==============================] - 0s 908us/step - loss: 0.0325 - accuracy: 1.0000
Epoch 86/250
10/10 [==============================] - 0s 698us/step - loss: 0.0188 - accuracy: 1.0000
Epoch 87/250
10/10 [==============================] - 0s 698us/step - loss: 0.1132 - accuracy: 0.9362
Epoch 88/250
10/10 [==============================] - 0s 699us/step - loss: 0.0367 - accuracy: 1.0000
Epoch 89/250
10/10 [==============================] - 0s 625us/step - loss: 0.0534 - accuracy: 0.9787
Epoch 90/250
10/10 [==============================] - 0s 700us/step - loss: 0.0434 - accuracy: 

10/10 [==============================] - 0s 698us/step - loss: 0.0049 - accuracy: 1.0000
Epoch 161/250
10/10 [==============================] - 0s 725us/step - loss: 0.0178 - accuracy: 1.0000
Epoch 162/250
10/10 [==============================] - 0s 781us/step - loss: 0.0185 - accuracy: 1.0000
Epoch 163/250
10/10 [==============================] - 0s 802us/step - loss: 0.0214 - accuracy: 1.0000
Epoch 164/250
10/10 [==============================] - 0s 805us/step - loss: 0.0310 - accuracy: 1.0000
Epoch 165/250
10/10 [==============================] - 0s 598us/step - loss: 0.0085 - accuracy: 1.0000
Epoch 166/250
10/10 [==============================] - 0s 798us/step - loss: 0.0895 - accuracy: 0.9787
Epoch 167/250
10/10 [==============================] - 0s 682us/step - loss: 0.0240 - accuracy: 1.0000
Epoch 168/250
10/10 [==============================] - 0s 698us/step - loss: 0.0060 - accuracy: 1.0000
Epoch 169/250
10/10 [==============================] - 0s 798us/step - loss: 0.0296 - a

10/10 [==============================] - 0s 700us/step - loss: 0.0122 - accuracy: 1.0000
Epoch 240/250
10/10 [==============================] - 0s 619us/step - loss: 0.0058 - accuracy: 1.0000
Epoch 241/250
10/10 [==============================] - 0s 799us/step - loss: 0.0668 - accuracy: 0.9787
Epoch 242/250
10/10 [==============================] - 0s 652us/step - loss: 0.0052 - accuracy: 1.0000
Epoch 243/250
10/10 [==============================] - 0s 716us/step - loss: 0.0063 - accuracy: 1.0000
Epoch 244/250
10/10 [==============================] - 0s 699us/step - loss: 0.0059 - accuracy: 1.0000
Epoch 245/250
10/10 [==============================] - 0s 598us/step - loss: 0.0149 - accuracy: 1.0000
Epoch 246/250
10/10 [==============================] - 0s 698us/step - loss: 0.0030 - accuracy: 1.0000
Epoch 247/250
10/10 [==============================] - 0s 598us/step - loss: 0.0194 - accuracy: 1.0000
Epoch 248/250
10/10 [==============================] - 0s 698us/step - loss: 0.0053 - a

In [64]:
from keras.models import load_model

model = load_model('chatbot_model.h5')

intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [65]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

In [66]:
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

In [67]:
def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

In [68]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

In [69]:
def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res

In [70]:
import tkinter as tk
from tkinter import *

In [71]:
def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)
    
    if msg != "":
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END,"You: "+msg+ '\n\n')
        ChatLog.config(foreground="#442265",font=("Tahoma",10))
        
        res = chatbot_response(msg)
        ChatLog.insert(END,"Bot: "+res+'\n\n')
        
        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)
        
base = tk.Tk()
base.title("Hello")
base.geometry("500x750")
base.resizable(width=False,height=False)

#Create Chat window
ChatLog = Text(base,bd=0,bg='white',height='8',width='50',font="Arial")

ChatLog.config(state=DISABLED)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(base,command=ChatLog.yview,cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

#Create Button to send message
sendbutton = Button(base,font=("Tahoma",10,'bold'),text="Send",width='12',height='5',bd=0,bg="#32de97",activebackground="#3c9d9b",fg='#ffffff',
                    command= send)
#Create the box to enter message
EntryBox = Text(base,bd=0,bg="white",width="29",height="5",font="Arial")

#Place all components on the screen
scrollbar.place(x=376,y=6,height=386)
ChatLog.place(x=6,y=6,height=386,width=370)
EntryBox.place(x=128,y=401,height=90,width=265)
sendbutton.place(x=6,y=401,height=90)



In [72]:
base.mainloop()